In [1]:
import boto3

In [2]:
session = boto3.session.Session()
client = session.client(
    service_name='dynamodb'
)

In [3]:
table_name = 'Orders'

(order_id, customer_id) will be the primary key and

status is GSI

Although this is not a good design, as order_id is unique for each customer. customer_id should be index instead

In [4]:
table = client.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'order_id', 'KeyType': 'HASH'},   # Partition key
            {'AttributeName': 'customer_id', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'order_id', 'AttributeType': 'S'},
            {'AttributeName': 'customer_id', 'AttributeType': 'S'},
            {'AttributeName': 'status', 'AttributeType': 'S'}
        ],
        GlobalSecondaryIndexes=[
            {
                'IndexName': 'StatusIndex',
                'KeySchema': [
                    {'AttributeName': 'status', 'KeyType': 'HASH'}
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                },
                'ProvisionedThroughput': {
                    'ReadCapacityUnits': 5,
                    'WriteCapacityUnits': 5
                }
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

In [5]:
# Status 
dynamodb = boto3.resource('dynamodb', region_name ='us-east-1')
table = dynamodb.Table(table_name)

print("Table status:", table.table_status)

Table status: CREATING


In [8]:

items = [
    {'order_id': 'o1', 'customer_id': 'c1', 'status': 'pending', 'amount': 100},
    {'order_id': 'o2', 'customer_id': 'c2', 'status': 'shipped', 'amount': 250},
    {'order_id': 'o3', 'customer_id': 'c1', 'status': 'delivered', 'amount': 75},
]

for item in items:
    table.put_item(Item=item)

print(f"Inserted {len(items)} items into {table_name}.")

response = table.scan()
print("Current items in table:")
for i in response['Items']:
    print(i)

Inserted 3 items into Orders.
Current items in table:
{'amount': Decimal('100'), 'customer_id': 'c1', 'order_id': 'o1', 'status': 'pending'}
{'amount': Decimal('75'), 'customer_id': 'c1', 'order_id': 'o3', 'status': 'delivered'}
{'amount': Decimal('250'), 'customer_id': 'c2', 'order_id': 'o2', 'status': 'shipped'}


## Query DynamoDB

In [9]:
from boto3.dynamodb.conditions import Key

result = table.query(
        KeyConditionExpression=Key('order_id').eq('o3')
    )
for i in result['Items']:
    print(i)

{'amount': Decimal('75'), 'order_id': 'o3', 'status': 'delivered', 'customer_id': 'c1'}


### Using GSI

In [10]:
result = table.query(
    IndexName='StatusIndex',
    KeyConditionExpression=Key('status').eq('pending')
)

for i in result['Items']:
    print(i)

{'amount': Decimal('100'), 'customer_id': 'c1', 'order_id': 'o1', 'status': 'pending'}


## Update record

In [12]:

# Update status for a given order_id
response = table.update_item(
    Key={
        'order_id': 'o1',
        'customer_id': 'c1',
    },
    UpdateExpression="SET #s = :new_status",
    ExpressionAttributeNames={
        '#s': 'status'  
    },
    ExpressionAttributeValues={
        ':new_status': 'delivered'
    },
    ReturnValues="UPDATED_NEW"  
)

print(response['Attributes'])

{'status': 'delivered'}


In [13]:
response = table.delete_item(Key = {'order_id': 'o3', 'customer_id': 'c1'})



In [14]:
response = table.delete()
